# Artist Recommendation API

## Rishabh Sharma

In [25]:
# Import packages
from flask import Flask, request, jsonify
import pandas as pd

# Initialize app
app = Flask(__name__) #default

# Read in the predictions file and round off the score to nearest decimal
predictions = pd.read_csv("Predictions.csv").dropna() 
predictions.score = round(predictions.score, 1)

# Read in the artists file and drop the colmumns not needed
artists = pd.read_table("artists.dat").dropna()
artists = artists.drop(["url", "pictureURL"], axis = 1)

# Combine both files to get artist names and change the data type of id column
predictions = predictions.merge(artists, on="id", how="left").sort_values(['UserID', 'score'], ascending=[True, False])
predictions = predictions.astype({'id': 'int64'})

## The application definiton
### Endpoint - one route /ratings/top - one HTTP verb = GET
@app.route("/ratings/top", methods=["GET"])
def top_ratings():
    ## read the parameters of the API call
    userId_str = request.args.get("userId") #parameters specification is always in string form
    try:
        userId = int(userId_str)
    except:
        return "'UserID' is required and should be an Integer."
        sys.exit("'UserID' is required and should be an Integer.")
       
    count_str = request.args.get("count")
    try:
        count = int(count_str)
    except:
        count = 5
    
    # filter predictions for the given UserID
    predict = predictions[predictions.UserID == userId].head(count)
    
    # select Artist ID, name and score;  and transform to list
    top_ratings = list(predict[["id", "name", "score"]].T.to_dict().values()) 
    
    # Return the result to the API
    return jsonify(top_ratings)

### Put endpoint online
if __name__ == '__main__':
    app.run(host='localhost', port=6000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:6000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Mar/2020 22:58:51] "GET /ratings/top?userId=2100&count=5 HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2020 22:59:03] "GET /ratings/top?userId=1900&count=5 HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2020 22:59:18] "GET /ratings/top?userId=1970&count=5 HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2020 22:59:23] "GET /ratings/top?userId=1971&count=5 HTTP/1.1" 200 -


#### NOTE:

#### The datebase for the API is built only for the users belonging to the Test set which have user IDs from 1662 to 2100. However, some of them get dropped due to NAs in recommendation.

#### For the Flask API call, please pick a user ID in the above range or from the Prediction.csv file.